# Intro
The aim of this file is to:
- Step 1: load a slice of wikipedia dataset from HuggingFace, in order to study its structure,
- Step 2: the same thing but from a local file,
- Step 3: store it in a parquet file to learn how to create a dataset.

## Step 1
Aim: load a slice of wikipedia dataset from HuggingFace

In [10]:
"""Imports for loading datasets"""

from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

In [59]:
configs = get_dataset_config_names('wikipedia', trust_remote_code=True)

print(configs[:10]) # Just to show some examples. We will use '20220301.fr'.

['20220301.aa', '20220301.ab', '20220301.ace', '20220301.ady', '20220301.af', '20220301.ak', '20220301.als', '20220301.am', '20220301.an', '20220301.ang']


In [60]:
ds_builder = load_dataset_builder('wikipedia', '20220301.fr', trust_remote_code=True)

In [61]:
print(ds_builder.info.description, '\n')
print(ds_builder.info.features)

Wikipedia dataset containing cleaned articles of all languages.
The datasets are built from the Wikipedia dump
(https://dumps.wikimedia.org/) with one split per language. Each example
contains the content of one full Wikipedia article with cleaning to strip
markdown and unwanted sections (references, etc.).
 

{'id': Value(dtype='string', id=None), 'url': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}


In [62]:
wikipedia = load_dataset('wikipedia', '20220301.fr', trust_remote_code=True, split='train', streaming=True)

In [63]:
# Create a slice of the first three columns of wikipedia dataset

count = 0
slice_wiki = []
for column in wikipedia:
    if count >= 3:
        break
    slice_wiki.append(column)
    count += 1

In [64]:
for item in slice_wiki:
    print(f"""id: {item['id']}
url: {item['url']}
title: {item['title']}
text: {item['text'][:200]}
""")

id: 3
url: https://fr.wikipedia.org/wiki/Antoine%20Meillet
title: Antoine Meillet
text: Paul Jules Antoine Meillet, né le  à Moulins (Allier) et mort le  à Châteaumeillant (Cher), est le principal linguiste français des premières décennies du . Il est aussi philologue.

Biographie 
D'ori

id: 7
url: https://fr.wikipedia.org/wiki/Alg%C3%A8bre%20lin%C3%A9aire
title: Algèbre linéaire
text: L’algèbre linéaire est la branche des mathématiques qui s'intéresse aux espaces vectoriels et aux transformations linéaires, formalisation générale des théories des systèmes d'équations linéaires.

Hi

id: 9
url: https://fr.wikipedia.org/wiki/Alg%C3%A8bre%20g%C3%A9n%C3%A9rale
title: Algèbre générale
text: L'algèbre générale, ou algèbre abstraite, est la branche des mathématiques qui porte principalement sur l'étude des structures algébriques et de leurs relations. L'appellation algèbre générale s'oppos



### Step 1 finished
Here we already have a general idea of a column in the wikipedia dataset. The "id" key doesn't mean much but I guess it is just a unique identifier for a column.

## Step 2
Aim: load a slice of wikipedia dataset from a local parquet file

In [65]:
local_parquet = '../../data/train-00035-of-00041.parquet' # This is the English parquet '20220301.fr'

wiki_local = load_dataset('parquet', data_files=local_parquet, streaming=True)

In [66]:
count = 0
slice_local = []
for column in wikipedia:
    if count >= 3:
        break
    slice_local.append(column)
    count += 1

In [67]:
for item in slice_local:
    print(f"""id: {item['id']}
url: {item['url']}
title: {item['title']}
text: {item['text'][:200]}
""")

id: 3
url: https://fr.wikipedia.org/wiki/Antoine%20Meillet
title: Antoine Meillet
text: Paul Jules Antoine Meillet, né le  à Moulins (Allier) et mort le  à Châteaumeillant (Cher), est le principal linguiste français des premières décennies du . Il est aussi philologue.

Biographie 
D'ori

id: 7
url: https://fr.wikipedia.org/wiki/Alg%C3%A8bre%20lin%C3%A9aire
title: Algèbre linéaire
text: L’algèbre linéaire est la branche des mathématiques qui s'intéresse aux espaces vectoriels et aux transformations linéaires, formalisation générale des théories des systèmes d'équations linéaires.

Hi

id: 9
url: https://fr.wikipedia.org/wiki/Alg%C3%A8bre%20g%C3%A9n%C3%A9rale
title: Algèbre générale
text: L'algèbre générale, ou algèbre abstraite, est la branche des mathématiques qui porte principalement sur l'étude des structures algébriques et de leurs relations. L'appellation algèbre générale s'oppos



### Step 2 finished

In [71]:
"""Imports to create a dataset"""

from datasets import Dataset

In [75]:
# Create our own dataset, with five random pages from wikipedia in Chinese

dataset_dict = {'id': [], 'url': [], 'title': [], 'text': []}

id_list = [1, 2, 3, 4, 5]
url_list = ['https://zh.wikipedia.org/wiki/Python', 
            'https://zh.wikipedia.org/wiki/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86',
            'https://zh.wikipedia.org/wiki/%E5%90%89%E5%A4%9A%C2%B7%E8%8C%83%E7%BD%97%E8%8B%8F%E5%A7%86',
            'https://zh.wikipedia.org/wiki/Linux',
            'https://zh.wikipedia.org/wiki/%E6%AD%BB%E4%BA%A1%E6%93%B1%E6%B7%BA']
title_list = ['Python', '自然语言处理', '吉多·范罗苏姆', 'Linux', '死亡搁浅']
text_list = ['Python（英国发音：/ˈpaɪθən/；美国发音：/ˈpaɪθɑːn/），是一种广泛使用的解释型、高级和通用的编程语言。',
             '自然语言处理（英语：Natural Language Processing，缩写作 NLP）是人工智能和语言学领域的分支学科。',
             '吉多·范罗苏姆（荷兰语：Guido van Rossum，1956年1月31日—），生于荷兰哈勒姆，计算机程序员，为Python程序设计语言的最初设计者及主要架构师。',
             'Linux是一种自由和开放源码的类UNIX操作系统。该操作系统的内核由林纳斯·托瓦兹在1991年10月5日首次发布，再加上用户空间的应用程序之后，就成为了Linux操作系统。',
             '《死亡搁浅》（日语：デス・ストランディング，英语：Death Stranding）是一款由小岛工作室开发，索尼互动娱乐发行的开放世界动作冒险游戏。']

dataset_dict['id'] = id_list
dataset_dict['url'] = url_list
dataset_dict['title'] = title_list
dataset_dict['text'] = text_list

In [76]:
print(dataset_dict)

{'id': [1, 2, 3, 4, 5], 'url': ['https://zh.wikipedia.org/wiki/Python', 'https://zh.wikipedia.org/wiki/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86', 'https://zh.wikipedia.org/wiki/%E5%90%89%E5%A4%9A%C2%B7%E8%8C%83%E7%BD%97%E8%8B%8F%E5%A7%86', 'https://zh.wikipedia.org/wiki/Linux', 'https://zh.wikipedia.org/wiki/%E6%AD%BB%E4%BA%A1%E6%93%B1%E6%B7%BA'], 'title': ['Python', '自然语言处理', '吉多·范罗苏姆', 'Linux', '死亡搁浅'], 'text': ['Python（英国发音：/ˈpaɪθən/；美国发音：/ˈpaɪθɑːn/），是一种广泛使用的解释型、高级和通用的编程语言。', '自然语言处理（英语：Natural Language Processing，缩写作 NLP）是人工智能和语言学领域的分支学科。', '吉多·范罗苏姆（荷兰语：Guido van Rossum，1956年1月31日—），生于荷兰哈勒姆，计算机程序员，为Python程序设计语言的最初设计者及主要架构师。', 'Linux是一种自由和开放源码的类UNIX操作系统。该操作系统的内核由林纳斯·托瓦兹在1991年10月5日首次发布，再加上用户空间的应用程序之后，就成为了Linux操作系统。', '《死亡搁浅》（日语：デス・ストランディング，英语：Death Stranding）是一款由小岛工作室开发，索尼互动娱乐发行的开放世界动作冒险游戏。']}


In [78]:
my_ds = Dataset.from_dict(dataset_dict)
print(my_ds[0])

{'id': 1, 'url': 'https://zh.wikipedia.org/wiki/Python', 'title': 'Python', 'text': 'Python（英国发音：/ˈpaɪθən/；美国发音：/ˈpaɪθɑːn/），是一种广泛使用的解释型、高级和通用的编程语言。'}


In [79]:
output_file_path = '../../data/my_ds.parquet'
my_ds.to_parquet(output_file_path, compression='snappy')

Creating parquet from Arrow format: 100%|███████████████████████████████████| 1/1 [00:00<00:00, 60.87ba/s]


1341

In [83]:
# See if it worked

cn_ds = load_dataset('parquet', split='train', data_files=output_file_path)

for item in cn_ds:
    print(item, '\n')

{'id': 1, 'url': 'https://zh.wikipedia.org/wiki/Python', 'title': 'Python', 'text': 'Python（英国发音：/ˈpaɪθən/；美国发音：/ˈpaɪθɑːn/），是一种广泛使用的解释型、高级和通用的编程语言。'} 

{'id': 2, 'url': 'https://zh.wikipedia.org/wiki/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86', 'title': '自然语言处理', 'text': '自然语言处理（英语：Natural Language Processing，缩写作 NLP）是人工智能和语言学领域的分支学科。'} 

{'id': 3, 'url': 'https://zh.wikipedia.org/wiki/%E5%90%89%E5%A4%9A%C2%B7%E8%8C%83%E7%BD%97%E8%8B%8F%E5%A7%86', 'title': '吉多·范罗苏姆', 'text': '吉多·范罗苏姆（荷兰语：Guido van Rossum，1956年1月31日—），生于荷兰哈勒姆，计算机程序员，为Python程序设计语言的最初设计者及主要架构师。'} 

{'id': 4, 'url': 'https://zh.wikipedia.org/wiki/Linux', 'title': 'Linux', 'text': 'Linux是一种自由和开放源码的类UNIX操作系统。该操作系统的内核由林纳斯·托瓦兹在1991年10月5日首次发布，再加上用户空间的应用程序之后，就成为了Linux操作系统。'} 

{'id': 5, 'url': 'https://zh.wikipedia.org/wiki/%E6%AD%BB%E4%BA%A1%E6%93%B1%E6%B7%BA', 'title': '死亡搁浅', 'text': '《死亡搁浅》（日语：デス・ストランディング，英语：Death Stranding）是一款由小岛工作室开发，索尼互动娱乐发行的开放世界动作冒险游戏。'} 

